## Imports and Setup

In [1]:
import os
import sys
import joblib
import subprocess

import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm.notebook import tqdm

sys.path.insert(0, "../utils/")
import pred_utils
import config
import geoutils
import eval_utils
import fusion_utils

import logging
logging.basicConfig(level=logging.INFO)
pd.set_option('mode.chained_assignment', None)

%load_ext autoreload
%autoreload 2

C:\Users\Issa\miniconda3\envs\GFDRR\lib\site-packages\groundingdino\models\GroundingDINO\ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


## Generate Predictions

In [46]:
name = 'LCA_08'
in_file = f'../data/rasters/ortho/ortho_LCA.tif'
bldg_file = f'../data/vectors/{name}.gpkg'

In [47]:
#bldgs = gpd.read_file(bldg_file).reset_index(drop=True)[['UID', 'geometry']].dropna()
bldgs = gpd.read_file(bldg_file)
bldgs = bldgs.reset_index(drop=True)
bldgs['UID'] = bldgs.index
bldgs = bldgs[['UID', 'geometry']]
print(bldgs.shape)
bldgs.head(3)

(578, 2)


,UID,geometry
0,0,"POLYGON ((710665.900 1523895.900, 710675.600 1..."
1,1,"POLYGON ((710696.200 1523891.700, 710698.900 1..."
2,2,"POLYGON ((710646.900 1523914.100, 710660.900 1..."


In [48]:
exp_config = '../configs/cnn/cnn-roof_type-efficientnetb0-RGB_DOM_LCA.yaml'
c = config.load_config(exp_config, prefix='.')
bldgs = pred_utils.predict_image(bldgs, in_file, exp_config, prefix='.')
bldgs.head(3)

INFO:root:Config: {'attribute': 'roof_type', 'data': 'RGB_DOM_LCA', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'efficientnetb0', 'pretrained': True, 'resampler': 'RandomOverSampler', 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None, 'config_name': 'cnn-roof_type-efficientnetb0-RGB_DOM_LCA', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}
INFO:root:Model file ../exp/cnn-roof_type-efficientnetb0-RGB_DOM_LCA\cnn-roof_type-efficientnetb0-RGB_DOM_LCA.pth successfully loaded.
100%|███████████████| 578/578 [01:50<00:00,  5.22it/s]                                                                 


,UID,geometry,ROOF_TYPE,ROOF_TYPE_PROB,NO_ROOF_PROB,GABLE_PROB,HIP_PROB,FLAT_PROB
0,0,"POLYGON ((710665.900 1523895.900, 710675.600 1...",HIP,0.946960,0.015347,0.016730,0.946960,0.020964
1,1,"POLYGON ((710696.200 1523891.700, 710698.900 1...",GABLE,0.949959,0.015253,0.949959,0.006668,0.028121
2,2,"POLYGON ((710646.900 1523914.100, 710660.900 1...",HIP,0.915768,0.034023,0.028584,0.915768,0.021626


In [49]:
n_classes = None
exp_config = '../configs/cnn/cnn-roof_material-resnet50-RGB_DOM.yaml'
if 'DOM' not in name: 
    n_classes = 4
    exp_config = '../configs/cnn/cnn-roof_material-efficientnetb0-RGB_LCA.yaml'
c = config.load_config(exp_config, prefix='.')
bldgs = pred_utils.predict_image(bldgs, in_file, exp_config, n_classes=n_classes, prefix='.')
bldgs.head(3)

INFO:root:Config: {'data': 'RGB_LCA', 'mode': 'RGB', 'attribute': 'roof_material', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'efficientnetb0', 'pretrained': True, 'resampler': 'RandomOverSampler', 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None, 'config_name': 'cnn-roof_material-efficientnetb0-RGB_LCA', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}
INFO:root:Model file ../exp/cnn-roof_material-efficientnetb0-RGB_LCA\cnn-roof_material-efficientnetb0-RGB_LCA.pth successfully loaded.
100%|███████████████| 578/578 [01:57<00:00,  4.90it/s]                                                                 


,UID,geometry,ROOF_TYPE,ROOF_TYPE_PROB,NO_ROOF_PROB,GABLE_PROB,HIP_PROB,FLAT_PROB,ROOF_MATERIAL,ROOF_MATERIAL_PROB,INCOMPLETE_PROB,HEALTHY_METAL_PROB,IRREGULAR_METAL_PROB,CONCRETE_CEMENT_PROB
0,0,"POLYGON ((710665.900 1523895.900, 710675.600 1...",HIP,0.946960,0.015347,0.016730,0.946960,0.020964,HEALTHY_METAL,0.911675,0.029389,0.911675,0.028507,0.030429
1,1,"POLYGON ((710696.200 1523891.700, 710698.900 1...",GABLE,0.949959,0.015253,0.949959,0.006668,0.028121,IRREGULAR_METAL,0.928734,0.024777,0.035255,0.928734,0.011234
2,2,"POLYGON ((710646.900 1523914.100, 710660.900 1...",HIP,0.915768,0.034023,0.028584,0.915768,0.021626,HEALTHY_METAL,0.857821,0.031317,0.857821,0.087956,0.022906


In [50]:
out_path = os.path.join(c['out_dir'])
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}.gpkg')
bldgs.to_file(out_file, driver='GPKG')

## Data Processing

In [60]:
files = [f'{out_path}LCA_0{i}.gpkg' for i in range(1,9)]
files = files + [out_path+'ortho_LCA.gpkg']
data = []
for file in files:
    subdata = gpd.read_file(file)
    data.append(subdata)
data = gpd.GeoDataFrame(pd.concat(data), crs=subdata.crs).reset_index(drop=True)
data['UID'] = data.index
data.to_file(out_path+'ortho_LCA2.gpkg', driver='GPKG')
data.head(3)

,UID,ROOF_TYPE,ROOF_TYPE_PROB,NO_ROOF_PROB,GABLE_PROB,HIP_PROB,FLAT_PROB,ROOF_MATERIAL,ROOF_MATERIAL_PROB,INCOMPLETE_PROB,HEALTHY_METAL_PROB,IRREGULAR_METAL_PROB,CONCRETE_CEMENT_PROB,geometry
0,0,GABLE,0.762585,0.017053,0.762585,0.042751,0.177611,HEALTHY_METAL,0.941553,0.039412,0.941553,0.003978,0.015058,"POLYGON ((718897.700 1554235.100, 718891.300 1..."
1,1,NO_ROOF,0.935998,0.935998,0.023369,0.022744,0.017888,INCOMPLETE,0.859907,0.859907,0.028522,0.074399,0.037172,"POLYGON ((718836.400 1554348.200, 718839.100 1..."
2,2,FLAT,0.465888,0.406486,0.084322,0.043304,0.465888,IRREGULAR_METAL,0.759435,0.043396,0.101301,0.759435,0.095868,"POLYGON ((718700.300 1554370.400, 718700.300 1..."


## Model Evaluation

In [ ]:
ytrue_file = '../data/vectors/building_footprints_annotated_COLIHAUT.gpkg'
ytrue = gpd.read_file(ytrue_file)[['UID', 'roof_type', 'roof_material', 'geometry']].dropna()
ytrue.roof_type = ytrue.roof_type.replace({'PYRAMID': 'HIP', 'HALF_HIP': 'HIP'})
print(f"Data dimensions: {ytrue.shape}")
print(ytrue.roof_type.value_counts())
print(ytrue.roof_material.value_counts())
ytrue.head(3)

In [ ]:
ypred_file = f'../output/{name}-drone-20230616.gpkg'
ypred = gpd.read_file(ypred_file)[['UID', 'roof_type', 'roof_material', 'geometry']].dropna()
print(f"Data dimensions: {ypred.shape}")
print(ypred.roof_type.value_counts())
print(ypred.roof_material.value_counts())
ypred.head(3)

In [ ]:
attribute = 'roof_type'
classes = geoutils.classes_dict[attribute]
cm = eval_utils.get_confusion_matrix(ytrue[attribute], ypred[attribute], classes)
eval_utils.evaluate(ytrue[attribute], ypred[attribute])

## Data Fusion

In [ ]:
filename = '../data/vectors/bldgs_ortho_DOM.gpkg'
bldgs = gpd.read_file(filename).reset_index(drop=True)[['UID', 'geometry']].dropna()
print(bldgs.shape)
bldgs.head(3)

In [ ]:
c = config.load_config("../configs/fusion/fusion_LR_embeds.yaml", prefix='.')
c = {key: '.' + value if 'config1' in key or 'config2' in key else value for key, value in c.items()}
print(c)

c1 = config.load_config(c['config1'], prefix='.')
classes = geoutils.get_classes_dict(c1['attribute'])
model1 = pred_utils.load_model(c1, classes=classes)
print(c1)

c2 = config.load_config(c['config2'], prefix='.')
model2 = pred_utils.load_model(c2, classes=classes)
print(c2)

In [ ]:
source1 = '../data/rasters/ortho/ortho_DOM.tif'
source2 = '../data/rasters/lidar/ndsm_DOM.tif'
data = fusion_utils.predict(bldgs, c1, c2, model1, model2, source1=source1, source2=source2)

In [ ]:
data.to_csv(os.path.join(c['out_dir'], 'ortho_DOM.csv'))

In [ ]:
features = fusion_utils.get_features(c, data)
model_file = os.path.join(
    c['exp_dir'], 
    c['config_name'], 
    c['mode'], 
    c['model'], 
    f"{c['config_name']}.pkl"
)
model_file

In [ ]:
model = joblib.load(model_file)
preds = model.predict(data[features])
probs = model.predict_proba(data[features])
probs_col = [f"{classes[index]}_PROB" for index in range(len(classes))]
probs = pd.DataFrame(probs, columns=probs_col)
probs.head(3)

In [ ]:
name = 'ortho_lidar_DOM'
bldgs[c1["attribute"]] = preds
bldgs[f"{c1['attribute']}_PROB"] = probs.max(axis=1)
results = gpd.GeoDataFrame(pd.concat([bldgs, probs], axis=1))
results.columns = [
    col.upper() if col != "geometry" else col for col in results.columns
]
out_file = os.path.join(c['out_dir'], f'{name}.gpkg')
results.to_file(out_file, driver='GPKG')